### Running a python server

A konduit python runner requires declaring and configuring a python pipeline step.
This pipeline step can either be directly passed python code or a python code path.

Optionally, one is also able to declare a python path to be passed and used in the python script.
In our examples, we have a pre installed anaconda and use a custom python path to introduce
custom dependencies.

An example configuration is below: 

```json
{
  "@type": "InferenceConfiguration",
  "pipelineSteps": [
    {
      "@type": "PythonPipelineStep",
      "inputColumnNames": {
        "default": [
          "img_path"
        ]
      },
      "inputNames": [
        "default"
      ],
      "inputSchemas": {
        "default": [
          "String"
        ]
      },
      "outputColumnNames": {
        "default": [
          "label",
          "proba"
        ]
      },
      "outputNames": [
        "default"
      ],
      "outputSchemas": {
        "default": [
          "String",
          "String"
        ]
      },
      "pythonConfigs": {
        "default": {
          "@type": "PythonConfig",
          "pythonCodePath": "/usr/share/input-data/exec.py",
          "pythonInputs": {
            "img_path": "STR"
          },
          "pythonOutputs": {
            "label": "STR",
            "proba": "STR"
          },
          "pythonPath": "/opt/conda/lib/python37.zip:/opt/conda/lib/python3.7:/opt/conda/lib/python3.7/lib-dynload:/opt/conda/lib/python3.7/site-packages:/opt/conda/lib/python3.7/site-packages/konduitserving-0.1-py3.7.egg:/opt/conda/lib/python3.7/site-packages/pandas-0.24.2-py3.7-linux-x86_64.egg:/opt/conda/lib/python3.7/site-packages/requests_toolbelt-0.9.1-py3.7.egg:/opt/conda/lib/python3.7/site-packages/pyarrow-0.13.0-py3.7-linux-x86_64.egg:/opt/conda/lib/python3.7/site-packages/numpy-1.16.4-py3.7-linux-x86_64.egg:/opt/conda/lib/python3.7/site-packages/pytz-2019.2-py3.7.egg"
        }
      }
    }
  ],
  "servingConfig": {
    "@type": "ServingConfig",
    "httpPort": 65535,
    "listenHost": "0.0.0.0",
    "logTimings": true,
    "parallelInferenceConfig": {
      "@type": "ParallelInferenceConfig",
      "workers": 1
    }
  }
}
```

In the above example, we are running the below code which uses keras applications, downloads resnet and runs inference
on a specified image path.

The above json was generated using the konduit sdk, see below for an example:



In [ ]:
from konduit.inference import *
from konduit.json_utils import json_with_type

import random
import json

input_names = ['default']
output_names = ['default']
port = 65535
parallel_inference_config = ParallelInferenceConfig(workers=1)
serving_config = ServingConfig(http_port=port,
                                   log_timings=True,
                                   parallel_inference_config=parallel_inference_config)

python_config = PythonConfig(
        python_path=python_path,
        python_code_path='"/usr/share/exec.py',
        python_inputs={'img_path': 'STR'},
        python_outputs={'label': 'STR','proba':'FLOAT'}
)

python_pipeline_step = PythonPipelineStep(input_names=input_names,
                                              output_names=output_names,  
                                           input_schemas=({'default': ['String']}),
                                           output_schemas=({'default': ['String','Float']}),
                                           input_column_names={'default': ['img_path']},
                                             output_column_names={'default': ['label','proba']},
                                              python_configs={'default': python_config})

inference = InferenceConfiguration(serving_config=serving_config,
                                       pipeline_steps=[python_pipeline_step])


json_config = json_with_type(inference)
print(json.dumps(json_config, indent=4, sort_keys=True))

In [ ]:
# Note the sys declaration here is to sync with the server's python run time.
import sys
sys.version = '3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21)\n[GCC 7.3.0]'

# Run the pythons cript
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import tensorflow as tf
import keras

# needed when dealing with multi threading
sess = tf.Session()
keras.backend.set_session(sess)

model = ResNet50(weights='imagenet')

#img_path = 'African_Bush_Elephant_1-783x1024.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
decoded = decode_predictions(preds, top=3)[0]
id,label,proba = decoded[0]
proba = str(proba)

### Input data

Of note here are a few things for configuration:
 
The python script has an absolute path directory at /usr/share/input-data. This is a directory that is mounted in to the server
from the host machine, take a look at each directory's docker-compose.yml
An example declaration for ease of reference:

```yaml
pipeline:
    image: konduit-serving-with-conda
    ports:
     - 65535:65535
    volumes:
     - ./input-data:/usr/share/input-data #here is the volume declaration
    entrypoint:
     - java
     - -Dorg.bytedeco.javacpp.logger.debug=true
     - -Dorg.bytedeco.javacpp.logger=slf4j
     - -Dai.konduit.serving.python.javacpp.path.append=none
     - -Dvertx.options.maxEventLoopExecuteTime=10000000000000
     - -cp
     - /srv/konduit.jar
     - KonduitServerMain
     - --configPath
     - /usr/share/input-data/config.json # mounted
```


Everything else are just application specific and are mostly used for debugging purposes and can be ignored by most users.

2. Look at the above configuration json again to see where else the /usr/share/input-data is used.


### Customization

To run your own python script, you need to setup a few things in the python step.

* Input names/output names: This can generally be left alone as "default". This is generally used for more complex problems like multi input graphs.

* Input schemas/Output Schemas: These are types of the variables for input and output. Available types can be found [here](https://github.com/KonduitAI/konduit/blob/master/konduit-serving-api/src/main/java/ai/konduit/serving/serving/SchemaType.java)

* Python types: Python input types/output types are more specific to python. Python types can be found [here](https://github.com/KonduitAI/konduit/blob/master/konduit-serving-python/src/main/java/ai/konduit/serving/util/python/PythonVariables.java#L49)

* Mapping names and schemas: The types and variable names of the input output variables should 
be in the same order.

* Ports: Note that we specify an http port above. The port will depend on the application, just make sure whatever you specify
will not be used by other applications.

* Custom python path: Go in to the python repl you want to use and type: 
```python
import sys
':'.join(sys.path)
```

for the correct value to put in the configuration.